## Import the necessary libraries

In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

## Function find_notebook

In [2]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [3]:
print(find_notebook('integrates_into_database(Matrix Factorization)'))

integrates_into_database(Matrix Factorization).ipynb


## Class NotebookLoader

In [4]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""

    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)

        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [5]:
# Init a notebook loader
notebook_loader = NotebookLoader()

In [6]:
plot_rec_sys = notebook_loader.load_module('plot_based_recommender')

importing Jupyter notebook from plot_based_recommender.ipynb


In [9]:
# Get the unique users in database
# users_id = users_df['id'].unique()
movie_titles = plot_rec_sys.movies_df['title'] # Update recommendation list for each user
failed = 0
for movie_title in movie_titles:
    print(movie_title)
    try:
        recommendations_numpy = plot_rec_sys.get_top_five_recommendations(movie_title)
    except ValueError as e:
        failed += 1
        continue
    recommendations = []
    for recommendation in recommendations_numpy:
        recommendations.append(int(recommendation))
    plot_rec_sys.similar_movies_collection.update_one(
     {"id": int(plot_rec_sys.get_original_movie_id(movie_title))},
     {"$set": {"similar_movies": recommendations}}
    )
print(failed)

Heavenly Creatures
Twelve Monkeys
Antonia's Line
The Net
Natural Born Killers
The Specialist
The House of the Spirits
Apur Sansar
Some Folks Call It a Sling Blade
Notorious
Spellbound
Mr. Smith Goes to Washington
Little Buddha
Live Nude Girls
Mission: Impossible
Vertigo
Roman Holiday
All About Eve
20,000 Leagues Under the Sea
Casino
Leaving Las Vegas
From Dusk Till Dawn
Pulp Fiction
Mr. Holland's Opus
Crimson Tide
GoldenEye
Taxi Driver
Casper
Strange Days
Faster, Pussycat! Kill! Kill!
Batman
Pather Panchali
Faces
To Be or Not to Be
Die Hard
The Shawshank Redemption
Disclosure
Milk Money
The Mask
Blown Away
Hard Target
Emma
Mrs. Doubtfire
The Philadelphia Story


KeyboardInterrupt: 